<a href="https://colab.research.google.com/github/Manos-Mak120/Makrakis-Manos/blob/main/Articles_Newsbomb_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import time
import random

from tqdm import tqdm

In [2]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    return BeautifulSoup(response.text, 'html.parser')

In [3]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    return BeautifulSoup(response.text, 'html.parser')
def get_article_elements_from_soup(soup, article_url):
    # title
    try:
        title = soup.find("h1").text.strip()
    except:
        title = ""

    # datetime
    try:
        datetime_str = soup.find("time")["datetime"]
        # Parse the datetime string and format it as 'YYYY-MM-DD'
        date_obj = datetime.fromisoformat(datetime_str.replace('Z', '+00:00'))
        date_str = date_obj.strftime('%Y-%m-%d')
        time_str = date_obj.strftime('%H:%M:%S') # You can keep time in HH:MM:SS format if needed
    except:
        date_str = ""
        time_str = ""


    # section
    try:
        section_div = soup.find("div", class_="main-category-block") or soup.find("div", class_="main-category")
        section = section_div.text.strip() if section_div else ""
    except:
        section = ""

    # author
    try:
      author_tag = soup.find("a", class_="author-name") or soup.find("span", class_="author-name")
      author = author_tag.text.strip() if author_tag else ""
    except:
        author = ""
    # excerpt
    try:
        excerpt_div = soup.find("div", class_="p.main-intro") or soup.find(itemprop="description") or soup.find("div", class_="nx-excerpt")
        excerpt = excerpt_div.text.strip() if excerpt_div else ""
    except:
        excerpt = ""

    # text body
    try:
      content_div = soup.find("div", class_="main-text story-fulltext")
      elements = content_div.find_all(["p", "h2", "h3", "ul", "ol", "li"]) if content_div else []
      text_body = " ".join(el.get_text(strip=True) for el in elements if el.get_text(strip=True))
    except:
        text_body = ""

    # feature image
    try:
        img_block = soup.find("div", class_="article-image") or soup.find("figure")
        feat_img_link = img_block.find("img")["src"] if img_block and img_block.find("img") else ""
    except:
        feat_img_link = ""

    # image caption
    try:
        caption_div = soup.find("div", class_="image-caption") or soup.find("figcaption")
        image_caption = caption_div.text.strip() if caption_div else ""
    except:
        image_caption = ""

    articleD = {
        "article_url": article_url,
        "title": title,
        "date": date_str,
        "time": time_str,
        "section": section,
        "author": author,
        "excerpt": excerpt,
        "text_body": text_body,
        "feat_img_link": feat_img_link,
        "feat_img_cation": image_caption
    }

    return articleD

In [4]:
def scrape_newsbomb_articles_pages(teasers_df):
  all_articles_dataL = []
  base_url = "https://www.newsbomb.gr"
  for idx, row in tqdm(teasers_df.iterrows(), total=teasers_df.shape[0], desc="Processing rows"):
    article_url_relative = row['article_url']
    article_url = base_url + article_url_relative
    soup = get_webpage_soup(article_url)
    if soup:
     articleD = get_article_elements_from_soup(soup, article_url)
     all_articles_dataL.append(articleD)
    delay = random.uniform(1, 3)
    time.sleep(delay)
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetshed {len(df)} articles.")
  return df

**Δοκιμή του scraper**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
trump_teasers_df = pd.read_csv("/content/drive/MyDrive/trump_df.csv")
trump_teasers_df.head()

,title,article_url,date,excerpt,section
0,Ο Τραμπ ανακοινώνει το «Golden Dome»: Νέα διασ...,/kosmos/story/1650832/o-tramp-anakoinonei-to-g...,4 ώρες,Ο στρατηγός Γκέτλιν αναλαμβάνει πιθανότατα την...,ΚΟΣΜΟΣ
1,Ο Έλον Μασκ αποσύρεται από τις πολιτικές δωρεέ...,/kosmos/story/1650825/o-elon-mask-aposyretai-a...,4 ώρες,Μετά από 290 εκατ. δολάρια σε δαπάνες για τον ...,ΚΟΣΜΟΣ
2,Ντόναλντ Τραμπ: Τα τρία σενάρια μετά το τηλεφώ...,/kosmos/story/1650713/donalnt-tramp-ta-tria-se...,12 ώρες,Τα υπέρ και τα κατά θα ζυγίσει ο Αμερικανός πρ...,ΚΟΣΜΟΣ
3,ΗΠΑ: Σάλος με τους αριθμούς «86 47» που κοινοπ...,/kosmos/story/1650706/ipa-salos-me-tous-arithm...,12 ώρες,Σάλο έχει προκαλέσει η ανάρτηση του πρώην διευ...,ΚΟΣΜΟΣ
4,Το Sesame Street «μετακομίζει» στο Netflix λόγ...,/kosmos/story/1650699/to-sesame-street-metakom...,12 ώρες,Το Netflix ανακοίνωσε ότι η 56η σεζόν της αγαπ...,ΚΟΣΜΟΣ


In [8]:
trump_teasers_df_test = trump_teasers_df.head(10)
trump_teasers_df_test

,title,article_url,date,excerpt,section
0,Ο Τραμπ ανακοινώνει το «Golden Dome»: Νέα διασ...,/kosmos/story/1650832/o-tramp-anakoinonei-to-g...,4 ώρες,Ο στρατηγός Γκέτλιν αναλαμβάνει πιθανότατα την...,ΚΟΣΜΟΣ
1,Ο Έλον Μασκ αποσύρεται από τις πολιτικές δωρεέ...,/kosmos/story/1650825/o-elon-mask-aposyretai-a...,4 ώρες,Μετά από 290 εκατ. δολάρια σε δαπάνες για τον ...,ΚΟΣΜΟΣ
2,Ντόναλντ Τραμπ: Τα τρία σενάρια μετά το τηλεφώ...,/kosmos/story/1650713/donalnt-tramp-ta-tria-se...,12 ώρες,Τα υπέρ και τα κατά θα ζυγίσει ο Αμερικανός πρ...,ΚΟΣΜΟΣ
3,ΗΠΑ: Σάλος με τους αριθμούς «86 47» που κοινοπ...,/kosmos/story/1650706/ipa-salos-me-tous-arithm...,12 ώρες,Σάλο έχει προκαλέσει η ανάρτηση του πρώην διευ...,ΚΟΣΜΟΣ
4,Το Sesame Street «μετακομίζει» στο Netflix λόγ...,/kosmos/story/1650699/to-sesame-street-metakom...,12 ώρες,Το Netflix ανακοίνωσε ότι η 56η σεζόν της αγαπ...,ΚΟΣΜΟΣ
5,FT: Εγκατέλειψε ο Ντόναλντ Τραμπ την Ουκρανία;...,/kosmos/story/1650645/ft-egkateleipse-o-donaln...,16 ώρες,Μετά από την πολυαναμενόμενη τηλεφωνική κλήση ...,ΚΟΣΜΟΣ
6,Ο Τραμπ εξομαλύνει τις σχέσεις των ΗΠΑ με τη Ρ...,/kosmos/story/1650634/o-tramp-eksomalynei-tis-...,18 ώρες,Η Ρωσία δηλώνει έτοιμη να συνεργαστεί με την Ο...,ΚΟΣΜΟΣ
7,Τραμπ: Η διάγνωση για τον καρκίνο του Μπάιντεν...,/kosmos/story/1650627/tramp-i-diagnosi-gia-ton...,22 ώρες,"Το σχόλιο του Ντόναλντ Τραμπ, για την κατάστασ...",ΚΟΣΜΟΣ
8,Ζελένσκι: Το Κίεβο δεν θα δεχθεί απόσυρση δυνά...,/kosmos/story/1650611/zelenski-to-kievo-den-th...,Πριν από 1 ημέρα,Η Ουκρανία δεν θα συμφωνήσει να αποσύρει τις έ...,ΚΟΣΜΟΣ
9,Τραμπ μετά την επικοινωνία με Πούτιν: Ρωσία κα...,/kosmos/story/1650587/tramp-meta-tin-epikoinon...,Πριν από 1 ημέρα,"Το Βατικανό, εκπροσωπούμενο από τον Πάπα, δήλω...",ΚΟΣΜΟΣ


In [9]:
articles_df = scrape_newsbomb_articles_pages(trump_teasers_df)
articles_df

Processing rows: 100%|██████████| 100/100 [05:11<00:00,  3.11s/it]


Scraping completed. Fetshed 100 articles.


,article_url,title,date,time,section,author,excerpt,text_body,feat_img_link,feat_img_cation
0,https://www.newsbomb.gr/kosmos/story/1650832/o...,Ο Τραμπ ανακοινώνει το «Golden Dome»: Νέα διασ...,2025-05-20,19:49:26,ΚΟΣΜΟΣ,Newsbomb,Ο στρατηγός Γκέτλιν αναλαμβάνει πιθανότατα την...,Ο πρόεδρος των ΗΠΑΝτόναλντ Τραμπαναμένεται την...,https://img.bbmd.gr/img/1260/max/82/2025/05/20...,Αφίσες για την προτεινόμενη αντιπυραυλική ασπί...
1,https://www.newsbomb.gr/kosmos/story/1650825/o...,Ο Έλον Μασκ αποσύρεται από τις πολιτικές δωρεέ...,2025-05-20,19:15:39,ΚΟΣΜΟΣ,Newsbomb,Μετά από 290 εκατ. δολάρια σε δαπάνες για τον ...,"ΟΈλον Μασκ, ο πλουσιότερος άνθρωπος στον κόσμο...",https://img.bbmd.gr/img/1260/max/82/2025/05/20...,Ο Elon Musk χαιρετάει καθώς ο πρόεδρος Ντόναλν...
2,https://www.newsbomb.gr/kosmos/story/1650713/d...,Ντόναλντ Τραμπ: Τα τρία σενάρια μετά το τηλεφώ...,2025-05-20,12:12:39,ΚΟΣΜΟΣ,Newsbomb,Τα υπέρ και τα κατά θα ζυγίσει ο Αμερικανός πρ...,Μετά από τηνπολυαναμενόμενη τηλεφωνική κλήση μ...,https://img.bbmd.gr/img/1260/max/82/2025/05/20...,O πρόεδρος Τραμπ κατεβαίνει από το Air Force O...
3,https://www.newsbomb.gr/kosmos/story/1650706/i...,ΗΠΑ: Οι αριθμοί «86 47» του πρώην διευθυντή το...,2025-05-20,11:57:47,ΚΟΣΜΟΣ,Newsbomb,Σάλο έχει προκαλέσει η ανάρτηση του πρώην διευ...,"Ο πρώην διευθυντής του FBI, Τζέιμς Κόμι,δεν αν...",https://img.bbmd.gr/img/1260/max/82/2025/05/20...,"O πρώην διευθυντής του FBI, Τζέιμς Κόμι\nAssoc..."
4,https://www.newsbomb.gr/kosmos/story/1650699/t...,Το Sesame Street «μετακομίζει» στο Netflix λόγ...,2025-05-20,11:22:55,ΚΟΣΜΟΣ,Newsbomb,Το Netflix ανακοίνωσε ότι η 56η σεζόν της αγαπ...,Το Sesame Street και τοNetflixκατέληξαν σε συμ...,https://img.bbmd.gr/img/1260/max/82/2025/05/20...,Oι ήρωες της αγαπημένης παιδικής σειράς Sesame...
...,...,...,...,...,...,...,...,...,...,...
95,https://www.newsbomb.gr/kosmos/story/1648622/t...,Τραμπ: Προανήγγειλε εκτελεστικό διάταγμα για τ...,2025-05-12,03:29:07,ΚΟΣΜΟΣ,Newsbomb,Σκοπός να μειωθούν οι τιμές των συνταγογραφούμ...,ΟΝτόναλντ Τραμπανακοίνωσε χθες Κυριακή πως σήμ...,https://img.bbmd.gr/img/1260/max/82/2025/05/07...,Copyright 2025 The Associated Press. All right...
96,https://www.newsbomb.gr/kosmos/story/1648621/t...,Τραμπ: Μνημειώδης είδηση η συμφωνία απελευθέρω...,2025-05-12,03:00:56,ΚΟΣΜΟΣ,Newsbomb,«Είμαι ευγνώμων σε όλους όσοι συνέβαλαν να έρθ...,ΟΝτόναλντ Τραμπχαρακτήρισε χθες Κυριακή «μνημε...,https://img.bbmd.gr/img/1260/max/82/2025/05/11...,AP.
97,https://www.newsbomb.gr/kosmos/story/1648569/r...,Ραντεβού στην Τουρκία δίνει ο Ζελένσκι - Θα πε...,2025-05-11,20:22:40,ΚΟΣΜΟΣ,Newsbomb,Η απάντηση ήρθε μετά τις αμερικανικές πιέσεις ...,"Απέδωσαν οι πιέσεις του Αμερικανού προέδρου,Ντ...",https://img.bbmd.gr/img/1260/max/82/2025/05/11...,Αρχείου AP
98,https://www.newsbomb.gr/kosmos/story/1648556/t...,Τραμπ: «Η Ουκρανία να συμφωνήσει άμεσα στην πρ...,2025-05-11,19:31:21,ΚΟΣΜΟΣ,Newsbomb,Νέα παρέμβαση του Αμερικανού προέδρου καθώς το...,Μήνυμα στον Ουκρανό πρόεδρο έστειλε οΝτόναλντ ...,https://img.bbmd.gr/img/1260/max/82/2025/05/11...,


In [10]:
articles_df['text_body'][0]  #έλεγχος ότι πήρε όλο το κείμενο

'Ο πρόεδρος των ΗΠΑΝτόναλντ Τραμπαναμένεται την Τρίτη να παρουσιάσει επίσημα το σχέδιο και τη διοικητική δομή τουφιλόδοξου προγράμματος Golden Dome— μιας διαστημικής αντιπυραυλικής ασπίδας που έχει κεντρική θέση στον στρατιωτικό του σχεδιασμό. Σύμφωνα με δύο Αμερικανούς αξιωματούχους, η ανακοίνωση θα γίνει στις 3 το μεσημέρι (ώρα Ουάσινγκτον) στον Λευκό Οίκο, παρουσία του υπουργού Άμυνας, Πιτ Χέγκσεθ. Ο στρατηγός Μάικλ Γκέτλιν, υπαρχηγός της Διαστημικής Διοίκησης των ΗΠΑ, αναμένεται να αναλάβει τη διοίκηση του έργου, το οποίο εκτιμάται ότι θα κοστίσει δεκάδες δισεκατομμύρια δολάρια και θα χρειαστεί χρόνια για να υλοποιηθεί. Πυραυλική άμυνα με δορυφόρους Το Golden Dome αποσκοπεί στη δημιουργία ενός εκτεταμένου δικτύου δορυφόρων που θα εντοπίζουν, θα παρακολουθούν και ενδεχομένως θα αναχαιτίζουν εχθρικούς πυραύλους προτού φτάσουν στο έδαφος των ΗΠΑ. Εμπνευσμένο από το ισραηλινό Iron Dome, το αμερικανικό σχέδιο προχωρά ένα βήμα παραπέρα, προβλέποντας όχι μόνο δορυφόρους εντοπισμού αλλά κα

In [11]:
articles_df['feat_img_link'][0]

'https://img.bbmd.gr/img/1260/max/82/2025/05/20/614cdda1-AP25132564068676.jpg?t=w7ncGPp_C_Bi37nQuxeU1g'